In [ ]:
!pip install gensim

In [ ]:
import gensim
from gensim.models import KeyedVectors
import gensim.downloader as api
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
import re

class WordEmbedder :
  def __init__(self, model_name: str) :
       print("Đang tải mô hình GloVe...")
       self.model = api.load(model_name)
       print("Mô hình đã tải xong!")

  def get_vector(self, word: str):
    if word in self.model :
      return self.model[word]
    else :
      raise ValueError(f"Từ '{word}' không có trong từ điển của model.")

  def get_similarity(self, word1: str, word2: str):
    vec1 = self.get_vector(word1)
    vec2 = self.get_vector(word2)
    if vec1 is None or vec2 is None:
            return None
    # Tính cosine similarity: (v1 . v2) / (||v1|| * ||v2||)
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    if norm1 == 0 or norm2 == 0:
            return 0.0
    similarity = dot_product / (norm1 * norm2)
    return similarity

  def get_most_similar(self, word: str, top_n: int = 10):
    vec = self.get_vector(word)
    if vec is None:
            return []

        # Tính cosine similarity giữa vector của từ đầu vào và tất cả vector trong từ điển
    similarities = []
    for vocab_word in self.model.index_to_key:
      vocab_vec = self.get_vector(vocab_word)
      sim = self.get_similarity(word, vocab_word)
      if sim is not None:
        similarities.append((vocab_word, sim))

    # Sắp xếp theo độ tương tự giảm dần và lấy top_n
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:top_n]

  def tokenize(self, text: str) -> list[str]:
      text = text.lower()
      pattern = r"\w+|[^\w\s]"
      tokens = re.findall(pattern, text)
      return tokens
  def embed_document(self, document: str):
    tokens = self.tokenize(document)
    vectors = []
    for token in tokens:
        vec = self.get_vector(token)
        if vec is not None:
            vectors.append(vec)
    if not vectors:
        return np.zeros(self.model.vector_size)
    return np.mean(vectors, axis=0)



In [ ]:

# Khởi tạo với model GloVe
embedder = WordEmbedder('glove-wiki-gigaword-50')
# Lấy vector của từ 'king'
print(embedder.get_vector('king')[:10])  # In 10 giá trị đầu của vector

Đang tải mô hình GloVe...
Mô hình đã tải xong!
[ 0.50451   0.68607  -0.59517  -0.022801  0.60046  -0.13498  -0.08813
  0.47377  -0.61798  -0.31012 ]


In [ ]:
# Tính tương đồng
print("Similarity between king and queen:", embedder.get_similarity('king', 'queen'))
print("Similarity between king and man:", embedder.get_similarity('king', 'man'))
# Tìm top K tương đồng với từ "computer"
print(embedder.get_most_similar('computer'))


Similarity between king and queen: 0.7839044
Similarity between king and man: 0.5309377
[('computer', 1.0), ('computers', 0.91650456), ('software', 0.8814993), ('technology', 0.8525559), ('electronic', 0.8125868), ('internet', 0.80604553), ('computing', 0.8026036), ('devices', 0.8016185), ('digital', 0.79917926), ('applications', 0.79127395)]


In [ ]:

embed_text = embedder.embed_document('The queen rules the country.')
print(embed_text)

[ 0.04564168  0.36530998 -0.55974334  0.04014383  0.09655549  0.15623933
 -0.33622834 -0.12495166 -0.01031508 -0.5006717   0.18690467  0.17482166
 -0.268985   -0.03096624  0.36686516  0.29983264  0.01397333 -0.06872118
 -0.3260683  -0.210115    0.16835399 -0.03151734 -0.06204716  0.04301083
 -0.06958768 -1.7792168  -0.54365396 -0.06104483 -0.17618     0.009181
  3.3916333   0.08742473 -0.4675417  -0.213435    0.02391887 -0.04470453
  0.20636833 -0.12902866 -0.28527132 -0.2431805  -0.3114423  -0.03833717
  0.11977985 -0.01418401 -0.37086335  0.22069354 -0.28848937 -0.36188802
 -0.00549529 -0.46997246]


#  Bonus Task: Training a Word2Vec Model from Scratch

In [ ]:
import re
from gensim.models import Word2Vec
import os

def tokenize(text: str) -> list:
    """Tokenizer từ Lab 1: Tách từ và dấu bằng regex."""
    text = text.lower()
    pattern = r"\w+|[^\w\s]"
    tokens = re.findall(pattern, text)
    return tokens

def read_data(file_path: str) -> list:
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    tokens = tokenize(text)
    return [tokens]

def main():
    # Bước 1: Đọc dữ liệu
    data_path = '/content/en_ewt-ud-train.txt'
    if not os.path.exists(data_path):
        print(f"Không tìm thấy file: {data_path}")
        return

    print("Đang đọc dữ liệu...")
    sentences = read_data(data_path)
    print(f"✅ Đã đọc {len(sentences[0])} tokens")

    # Bước 2: Huấn luyện mô hình Word2Vec
    print("Đang huấn luyện Word2Vec...")
    model = Word2Vec(
        sentences=sentences,
        vector_size=10,
        window=5,
        min_count=2,   # ⚠️ Nên thêm để bỏ từ xuất hiện ít
        sg=1,          # ⚙️ Dùng Skip-gram cho ngữ nghĩa tốt hơn
        epochs=30
    )

    # Bước 3: Lưu mô hình
    os.makedirs("results", exist_ok=True)  # ⚠️ Thêm dòng này để tránh lỗi thư mục
    model_path = 'results/word2vec_ewt.model'
    model.save(model_path)
    print(f"✅ Đã lưu mô hình vào {model_path}")

    # Bước 4: Demo sử dụng
    print("\nDemo sử dụng mô hình:")
    word = "woman"
    if word in model.wv:
        print(f"Các từ tương tự với '{word}':")
        similar_words = model.wv.most_similar(word, topn=5)
        for w, sim in similar_words:
            print(f"  {w}: {sim:.4f}")
    else:
        print(f"Từ '{word}' không có trong từ điển.")

    analogy_words = ['king', 'woman', 'man']
    if all(w in model.wv for w in analogy_words):
        print("\nGiải analogy: king - man + woman = ?")
        result = model.wv.most_similar(
            positive=['king', 'woman'],
            negative=['man'],
            topn=5
        )
        for w, sim in result:
            print(f"  {w}: {sim:.4f}")
    else:
        print("\nKhông thể giải analogy vì thiếu từ trong từ điển.")

if __name__ == "__main__":
    main()


Đang đọc dữ liệu...
✅ Đã đọc 217306 tokens
Đang huấn luyện Word2Vec...
✅ Đã lưu mô hình vào results/word2vec_ewt.model

Demo sử dụng mô hình:
Các từ tương tự với 'woman':
  acquire: 0.9336
  rachel: 0.8780
  toy: 0.8627
  liquidweb: 0.8610
  268: 0.8548

Giải analogy: king - man + woman = ?
  acquire: 0.9056
  toy: 0.8872
  hybrid: 0.8863
  rachel: 0.8400
  circulate: 0.8366
